# Music and Functions

In [ ]:
from wav_library import *
from math import sin, pi
from IPython.display import Audio

## Wave Generator

So fare we have been creating sine waves like this:

In [ ]:
fs = 44100.0   # Sampling Rate 
f0 = 440.0     # Fundamental frequency
duration = 1.0 # in seconds

delta = 2.0 * pi * f0 / fs # how much does the phase change between samples

sine_wave = [sin(delta * i) for i in range(int(duration*fs))]

Which can be a little tedious to write, every isngle time. There is also more chance of making a mistake.

What if instead we consolidate all of the above in order to write it in one line like:

```py
square_wave = make_note(freq=440, sample_rate=44100, duration=1.0, shape='square')
```

We can even make some default values so that we don't have to write them out explicitly.

For this we create our own function

In [ ]:
def make_note(freq, duration, sample_rate=44100, shape='sine'):
    
    f0 = freq
    fs = sample_rate
    delta = 2.0 * pi * f0 / fs
    output = [sin(delta * i) for i in range(int(duration*fs))]
    
    return output

In [ ]:
wave = make_note(440, 0.05, shape='triangle')
Audio(data=wave, rate=fs)

Now we can make all kinds of changes to our function to give it new options. Or maybe we find a better way of doing something and all we have to do is change the contents of function and the rest of our code will remain unchanged.

We could add in the option for different wave shapes

In [ ]:
def make_note(freq, duration, sample_rate=44100, shape='sine', num_harmonics = 10):
    
    f0 = freq
    fs = sample_rate
    num_samples = int(duration*fs)
    
    output = []
    
    if (shape == 'sine'):
        delta = 2.0 * pi * f0 / fs
        output = [sin(delta * i) for i in range(int(duration*fs))]
    elif (shape == 'square'):
        for i in range(num_samples):
            output.append(0.0)
            for k in range(1,num_harmonics,2):
                gain = 1.0 / (f0 * (k+1))
                delta = 2.0 * pi * f0 * k / fs 
                output[i] += gain*sin(delta * i)

        maximum = max(output)

        for i in range(num_samples):
            output[i] *= 1.0 / maximum

    elif (shape == 'triangle'):
        num_harmonics = 10

        for i in range(num_samples):
            output.append(0.0)
            for n in range(0,num_harmonics):
                gain = ((-1.0)**(n)) / ((2*n+1)**2)
                delta = 2.0 * pi * f0 * (2*n+1) / fs 
                output[i] += gain*sin(delta * i)

        maximum = max(output)

        for i in range(num_samples):
            output[i] *= 1.0 / maximum
    
    return output

In [ ]:
def make_rest(duration, sample_rate=44100):    
    num_samples = int(duration*fs)
    return [0.0 for x in range(num_samples)]

Instead of having to write:

In [ ]:
melody = []

sampling_rate = 44100
beats_per_minute = 240
beats = 8
beat_length = 60 / beats_per_minute
note_length_seconds = beat_length / 2
note_length_samples = int(note_length_seconds * sampling_rate)

rest = [0.0 for x in range(note_length_samples)]
a_major = [440.0,494.0,554.0,622.0,659.0,740.0,831.0,880.0]

notes = []

for freq in a_major:
    delta = 2.0 * pi * freq / sampling_rate
    note = [sin(delta * i) for i in range(note_length_samples)]
    notes.append(note)

for i in range(beats):
    melody.extend(notes[i])
    melody.extend(rest)

with our new functions we can now write:

In [ ]:
a_major = [440.0,494.0,554.0,622.0,659.0,740.0,831.0,880.0]
note_length = 0.1
melody = []


for freq in a_major:
    melody.extend(make_note(freq, note_length))
    melody.extend(make_rest(note_length))

Audio(data=melody, rate=fs)

In [ ]:
score = [1,2,3,1,3,1,3,2,3,4,4,3,2,4,3,4,5,3,5,3,5,4,5,6,6,5,4,6,5,1,2,3,4,4,5,6,6,2,3,4,5,6,7,7,3,4,5,6,7,8,8,7,6,4,7,5,8]
beats = [2.6,8,2.6,8,4,4,2,2.6,8,8,8,8,8,1,
         2.6,8,2.6,8,4,4,2,2.6,8,8,8,8,8,1,
         2.6,8,8,8,8,8,1,2.6,8,8,8,8,8,1,
         2.6,8,8,8,8,8,1.6,8,8,4,4,4,4,4,1]

melody = []

beats_per_minute = 240
beat_length = 60 / beats_per_minute

for note,rhythm in zip(score,beats):
    note_length = (beat_length * 4 / rhythm)
    melody.extend(make_note(a_major[note-1], note_length))
    melody.extend(make_rest(0.02))

Audio(data=melody, rate=fs)

`2.6`, `1.6`, these aren't intuitive ways to write a rhythm. What if instead we could use something like `4.` to represent a doted note.

`note_length = make_rhythm(rhythm='2.', bpm=120)`

In [ ]:
def make_rhythm(rhythm, bpm=120):
    is_dotted = rhythm.endswith('.')
    rhythm_int = int(rhythm[:-1]) if is_dotted else int(rhythm)
    beat_length = 60 / beats_per_minute
    note_length = (beat_length * 4 / rhythm_int)
    
    if is_dotted:
        note_length *= 1.5
    
    return note_length

In [ ]:
score = [1,2,3,1,3,1,3,2,3,4,4,3,2,4,3,4,5,3,5,3,5,4,5,6,6,5,4,6,5,1,2,3,4,4,5,6,6,2,3,4,5,6,7,7,3,4,5,6,7,8,8,7,6,4,7,5,8]
beats = ['4.','8','4.','8','4','4','2','4.','8','8','8','8','8','1',
         '4.','8','4.','8','4','4','2','4.','8','8','8','8','8','1',
         '4.','8','8','8','8','8','1','4.','8','8','8','8','8','1',
         '4.','8','8','8','8','8','2.','8','8','4','4','4','4','4','1']
         
melody = []

beats_per_minute = 120

for note,rhythm in zip(score,beats):
    note_length = make_rhythm(rhythm, bpm=beats_per_minute)
    melody.extend(make_note(a_major[note-1], note_length, shape='triangle'))
    melody.extend(make_rest(0.02))

Audio(data=melody, rate=fs)

Does not sound quite right as the scale actually has some accidentals.
Apply the same logic, add a `#` or a `b` to make an accidental.


In [ ]:
def make_accidental(freq,accidental='#'):    
    return 2 ** ((1 if accidental == '#' else -1) / 12.0) * freq;

def make_pitch(note_num):
    
    
    major_scale = [261.63,293.66,329.63,349.23,392,440,493.88,523.25]
    is_accidental = note_num.endswith('#') or note_num.endswith('b')

    note_int = int(note_num[0])-1
    note_freq = major_scale[note_int]
    
    if is_accidental:
        note_freq = make_accidental(note_freq,note_num[-1])
        
    return note_freq

In [ ]:
score = ['1','2','3','1','3','1','3','2','3','4','4','3','2','4',
         '3','4','5','3','5','3','5','4','5','6','6','5','4','6',
         '5','1','2','3','4','5','6','6','2','3','4#','5','6','7',
         '7','3','4#','5#','6','7','8','7','7b','6','4','7','5','8']
beats = ['4.','8','4.','8','4','4','2','4.','8','8','8','8','8','1',
         '4.','8','4.','8','4','4','2','4.','8','8','8','8','8','1',
         '4.','8','8','8','8','8','1','4.','8','8','8','8','8','1',
         '4.','8','8','8','8','8','2.','8','8','4','4','4','4','4','1']
         
melody = []

beats_per_minute = 180

for note,rhythm in zip(score,beats):
    note_length = make_rhythm(rhythm, bpm=beats_per_minute)
    melody.extend(make_note(make_pitch(note), note_length, shape='triangle'))
    melody.extend(make_rest(0.02))

Audio(data=melody, rate=fs)

## Fade in

## Fade Out

## ADSR

## Helper Functions

## Piano Roll

## Signal Processing

## Modulation

### Amplitude Modulation

### Frequency Modulation

## Filters